# Network in Network (NiN)

:label:`sec_nin`


LeNet, AlexNet, and VGG all share a common design pattern:
extract features exploiting *spatial* structure
via a sequence of convolutions and pooling layers
and then post-process the representations via fully-connected layers.
The improvements upon LeNet by AlexNet and VGG mainly lie
in how these later networks widen and deepen these two modules.
Alternatively, one could imagine using fully-connected layers
earlier in the process.
However, a careless use of dense layers might give up the
spatial structure of the representation entirely,
Network in Network (NiN) blocks offer an alternative.
They were proposed in :cite:`Lin.Chen.Yan.2013` based on a very simple insight---to
use an MLP on the channels for each pixel separately.

## NiN Blocks

Recall that the inputs and outputs of convolutional layers
consist of four-dimensional arrays with axes
corresponding to the batch, channel, height, and width.
Also recall that the inputs and outputs of fully-connected layers
are typically two-dimensional arrays corresponding to the batch, and features.
The idea behind NiN is to apply a fully-connected layer
at each pixel location (for each height and  width).
If we tie the weights across each spatial location,
we could think of this as a $1\times 1$ convolutional layer
(as described in :numref:`sec_channels`)
or as a fully-connected layer acting independently on each pixel location.
Another way to view this is to think of each element in the spatial dimension
(height and width) as equivalent to an example
and the channel as equivalent to a feature.
:numref:`fig_nin` illustrates the main structural differences
between NiN and AlexNet, VGG, and other networks.

![The figure on the left shows the network structure of AlexNet and VGG, and the figure on the right shows the network structure of NiN. ](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/nin.svg)

:width:`600px`


:label:`fig_nin`



The NiN block consists of one convolutional layer
followed by two $1\times 1$ convolutional layers that act as
per-pixel fully-connected layers with ReLU activations.
The convolution width of the first layer is typically set by the user.
The subsequent widths are fixed to $1 \times 1$.

In [1]:
%use @file[../djl-pytorch.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.Training
import ai.djl.metric.Metrics

fun getLong(nm: String, n: Long): Long {
    val name = System.getProperty(nm)
    return if (null == name) n.toLong() else name.toLong()
}

// %load ../utils/djl-imports
// %load ../utils/plot-utils
// %load ../utils/DataPoints.java
// %load ../utils/Training.java
// %load ../utils/Accumulator.java

In [2]:
import ai.djl.basicdataset.cv.classification.*;
import ai.djl.metric.*;
// import org.apache.commons.lang3.ArrayUtils;

In [3]:
// setting the seed for demonstration purpose. You can remove it when you run the notebook
Engine.getInstance().setRandomSeed(5555);

fun  niNBlock(numChannels: Int, kernelShape: Shape, 
                     strideShape:Shape , paddingShape: Shape ): SequentialBlock{
    
    val tempBlock = SequentialBlock();
    
    tempBlock.add(Conv2d.builder()
              .setKernelShape(kernelShape)
              .optStride(strideShape)
              .optPadding(paddingShape)
              .setFilters(numChannels)
              .build())
        .add(Activation::relu)
        .add(Conv2d.builder()
              .setKernelShape(Shape(1, 1))
              .setFilters(numChannels)
              .build())
        .add(Activation::relu)
        .add(Conv2d.builder()
              .setKernelShape(Shape(1, 1))
              .setFilters(numChannels)
              .build())
        .add(Activation::relu); 
    
    return tempBlock;
}

## NiN Model

The original NiN network was proposed shortly after AlexNet
and clearly draws some inspiration.
NiN uses convolutional layers with window shapes
of $11\times 11$, $5\times 5$, and $3\times 3$,
and the corresponding numbers of output channels are the same as in AlexNet. Each NiN block is followed by a maximum pooling layer
with a stride of 2 and a window shape of $3\times 3$.

Once significant difference between NiN and AlexNet
is that NiN avoids dense connections altogether.
Instead, NiN uses an NiN block with a number of output channels equal to the number of label classes, followed by a *global* average pooling layer,
yielding a vector of [logits](https://en.wikipedia.org/wiki/Logit).
One advantage of NiN's design is that it significantly
reduces the number of required model parameters.
However, in practice, this design sometimes requires
increased model training time.

In [4]:
val block = SequentialBlock();

block.add(niNBlock(96, Shape(11, 11), Shape(4, 4), Shape(0, 0)))
     .add(Pool.maxPool2dBlock(Shape(3, 3), Shape(2, 2)))
     .add(niNBlock(256, Shape(5, 5), Shape(1, 1), Shape(2, 2)))
     .add(Pool.maxPool2dBlock(Shape(3, 3), Shape(2, 2)))
     .add(niNBlock(384, Shape(3, 3), Shape(1, 1), Shape(1, 1)))
     .add(Pool.maxPool2dBlock(Shape(3, 3), Shape(2, 2)))
     .add(Dropout.builder().optRate(0.5f).build())
     // There are 10 label classes
     .add(niNBlock(10, Shape(3, 3), Shape(1, 1), Shape(1, 1)))
     // The global average pooling layer automatically sets the window shape
     // to the height and width of the input
     .add(Pool.globalAvgPool2dBlock())
     // Transform the four-dimensional output into two-dimensional output
     // with a shape of (batch size, 10)
     .add(Blocks.batchFlattenBlock());

SequentialBlock {
	SequentialBlock {
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
	}
	maxPool2d
	SequentialBlock {
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
	}
	maxPool2d
	SequentialBlock {
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
	}
	maxPool2d
	Dropout
	SequentialBlock {
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
		Conv2d
		LambdaBlock
	}
	globalAvgPool2d
	batchFlatten
}

We create a data example to see the output shape of each block.

In [5]:
val lr = 0.1f;
val model = Model.newInstance("cnn");
model.setBlock(block);

val loss = Loss.softmaxCrossEntropyLoss();

val lrt = Tracker.fixed(lr);
val sgd = Optimizer.sgd().setLearningRateTracker(lrt).build();

val config = DefaultTrainingConfig(loss).optOptimizer(sgd) // Optimizer (loss function)
                .optDevices(Engine.getInstance().getDevices(1)) // single GPU
                .addEvaluator(Accuracy()) // Model Accuracy
                .addTrainingListeners(*TrainingListener.Defaults.logging()); // Logging

val trainer = model.newTrainer(config);

val manager = NDManager.newBaseManager();
val X = manager.randomUniform(0f, 1.0f, Shape(1, 1, 224, 224));
trainer.initialize(X.getShape());

var currentShape = X.getShape();

for (i in 0 until block.getChildren().size()) {

    val newShape = block.getChildren().get(i).getValue().getOutputShapes(arrayOf<Shape>(currentShape))
    currentShape = newShape[0];
    println(block.getChildren().get(i).getKey() + " layer output : " + currentShape);
}

01SequentialBlock layer output : (1, 96, 54, 54)
02LambdaBlock layer output : (1, 96, 26, 26)
03SequentialBlock layer output : (1, 256, 26, 26)
04LambdaBlock layer output : (1, 256, 12, 12)
05SequentialBlock layer output : (1, 384, 12, 12)
06LambdaBlock layer output : (1, 384, 5, 5)
07Dropout layer output : (1, 384, 5, 5)
08SequentialBlock layer output : (1, 10, 5, 5)
09LambdaBlock layer output : (1, 10)
10LambdaBlock layer output : (1, 10)


## Data Acquisition and Training

As before we use Fashion-MNIST to train the model.
NiN's training is similar to that for AlexNet and VGG,
but it often uses a larger learning rate.

In [6]:
val batchSize = 128;
val numEpochs = Integer.getInteger("MAX_EPOCH", 10);

//double[] trainLoss;
//double[] testAccuracy;
//double[] epochCount;
//double[] trainAccuracy;

val epochCount = IntArray(numEpochs) { it + 1 }

val trainIter = FashionMnist.builder()
        .addTransform(Resize(224))
        .addTransform(ToTensor())
        .optUsage(Dataset.Usage.TRAIN)
        .setSampling(batchSize, true)
        .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build();

val testIter = FashionMnist.builder()
        .addTransform(Resize(224))
        .addTransform(ToTensor())
        .optUsage(Dataset.Usage.TEST)
        .setSampling(batchSize, true)
        .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build();

trainIter.prepare();
testIter.prepare();

In [7]:
val evaluatorMetrics = mutableMapOf<String, DoubleArray>();
val avgTrainTimePerEpoch = Training.trainingChapter6(trainIter, testIter, numEpochs, trainer, evaluatorMetrics);

Training:    100% |████████████████████████████████████████| Accuracy: 0.13, SoftmaxCrossEntropyLoss: 2.26
Validating:  100% |████████████████████████████████████████|████              ||██████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.36, SoftmaxCrossEntropyLoss: 1.72██                        | Accuracy: 0.24, SoftmaxCrossEntropyLoss: 1.99   | Accuracy: 0.26, SoftmaxCrossEntropyLoss: 1.93
Validating:  100% |████████████████████████████████████████|                         |��███ |
Training:     72% |█████████████████████████████           | Accuracy: 0.51, SoftmaxCrossEntropyLoss: 1.41                                     | Accuracy: 0.36, SoftmaxCrossEntropyLoss: 1.72

The Loss became NaN, try reduce learning rate,add clipGradient option to your optimizer, check input data and loss calculation.
ai.djl.TrainingDivergedException: The Loss became NaN, try reduce learning rate,add clipGradient option to your optimizer, check input data and loss calculation.
	at ai.djl.training.listener.DivergenceCheckTrainingListener.onTrainingBatch(DivergenceCheckTrainingListener.java:27)
	at ai.djl.training.EasyTrain.lambda$trainBatch$3(EasyTrain.java:115)
	at java.base/java.util.ArrayList.forEach(ArrayList.java:1541)
	at ai.djl.training.Trainer.notifyListeners(Trainer.java:271)
	at ai.djl.training.EasyTrain.trainBatch(EasyTrain.java:115)
	at ai.djl.training.EasyTrain.fit(EasyTrain.java:58)
	at Line_9$Training.trainingChapter6(Line_9.jupyter-kts:128)
	at Line_588.<init>(Line_588.jupyter-kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeCo

In [8]:
trainLoss = evaluatorMetrics.get("train_epoch_SoftmaxCrossEntropyLoss");
trainAccuracy = evaluatorMetrics.get("train_epoch_Accuracy");
testAccuracy = evaluatorMetrics.get("validate_epoch_Accuracy");

print("loss %.3f,".format(trainLoss[numEpochs - 1]))
print(" train acc %.3f,".format(trainAccuracy[numEpochs - 1]))
print(" test acc %.3f\n".format(testAccuracy[numEpochs - 1]))
print("%.1f examples/sec".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10, 9))))
println()

Line_590.jupyter-kts (1:1 - 10) Unresolved reference: trainLoss
Line_590.jupyter-kts (2:1 - 14) Unresolved reference: trainAccuracy
Line_590.jupyter-kts (3:1 - 13) Unresolved reference: testAccuracy
Line_590.jupyter-kts (5:27 - 36) Unresolved reference: trainLoss
Line_590.jupyter-kts (6:33 - 46) Unresolved reference: trainAccuracy
Line_590.jupyter-kts (7:33 - 45) Unresolved reference: testAccuracy
Line_590.jupyter-kts (8:86 - 88) The integer literal does not conform to the expected type Double
Line_590.jupyter-kts (8:90 - 91) The integer literal does not conform to the expected type Double

![Contour Gradient Descent.](https://d2l-java-resources.s3.amazonaws.com/img/nin-plot.png)

String[] lossLabel = new String[trainLoss.length + testAccuracy.length + trainAccuracy.length];

Arrays.fill(lossLabel, 0, trainLoss.length, "train loss");
Arrays.fill(lossLabel, trainAccuracy.length, trainLoss.length + trainAccuracy.length, "train acc");
Arrays.fill(lossLabel, trainLoss.length + trainAccuracy.length,
                trainLoss.length + testAccuracy.length + trainAccuracy.length, "test acc");

Table data = Table.create("Data").addColumns(
            DoubleColumn.create("epoch", ArrayUtils.addAll(epochCount, ArrayUtils.addAll(epochCount, epochCount))),
            DoubleColumn.create("metrics", ArrayUtils.addAll(trainLoss, ArrayUtils.addAll(trainAccuracy, testAccuracy))),
            StringColumn.create("lossLabel", lossLabel)
);

render(LinePlot.create("", data, "epoch", "metrics", "lossLabel"),"text/html");

In [9]:
// String[] lossLabel = new String[trainLoss.length + testAccuracy.length + trainAccuracy.length];

val trainLossLabel =  Array<String>(trainLoss!!.size) { "train loss" }
val trainAccLabel = Array<String>(trainLoss!!.size) { "train acc" }
val testAccLabel = Array<String>(trainLoss!!.size) { "test acc" }
val data = mapOf<String, Any>(
      "label" to trainLossLabel + trainAccLabel + testAccLabel,
      "epoch" to epochCount + epochCount + epochCount,
      "metrics" to trainLoss!! + trainAccuracy!! + testAccuracy!!
)

var plot = letsPlot(data)
plot += geomLine { x = "epoch" ; y = "metrics" ; color = "label"}
plot + ggsize(700, 500)

Line_592.jupyter-kts (3:37 - 46) Unresolved reference: trainLoss
Line_592.jupyter-kts (4:35 - 44) Unresolved reference: trainLoss
Line_592.jupyter-kts (5:34 - 43) Unresolved reference: trainLoss
Line_592.jupyter-kts (9:20 - 29) Unresolved reference: trainLoss
Line_592.jupyter-kts (9:34 - 47) Unresolved reference: trainAccuracy
Line_592.jupyter-kts (9:52 - 64) Unresolved reference: testAccuracy

## Summary

* NiN uses blocks consisting of a convolutional layer and multiple $1\times 1$ convolutional layer. This can be used within the convolutional stack to allow for more per-pixel nonlinearity.
* NiN removes the fully connected layers and replaces them with global average pooling (i.e., summing over all locations) after reducing the number of channels to the desired number of outputs (e.g., 10 for Fashion-MNIST).
* Removing the dense layers reduces overfitting. NiN has dramatically fewer parameters.
* The NiN design influenced many subsequent convolutional neural networks designs.

## Exercises

1. Tune the hyper-parameters to improve the classification accuracy.
1. Why are there two $1\times 1$ convolutional layers in the NiN block? Remove one of them, and then observe and analyze the experimental phenomena.
1. Calculate the resource usage for NiN
    * What is the number of parameters?
    * What is the amount of computation?
    * What is the amount of memory needed during training?
    * What is the amount of memory needed during inference?
1. What are possible problems with reducing the $384 \times 5 \times 5$ representation to a $10 \times 5 \times 5$ representation in one step?
